![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-3-3-8b-instruct` to perform chat conversation with control roles

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for control roles available in IBM Granite models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goal

The purpose of this notebook is to demonstrate how to use control roles available in IBM Granite models.

## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install dependencies

In [1]:
%pip install -U ibm-watsonx-ai | tail -n 1

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Define the project ID
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id and hit enter: ")

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


Specify the `model_id` of the model you will use for the chat with tools.

In [4]:
model_id = "ibm/granite-3-3-8b-instruct"

### Define the model parameters

You might need to adjust model parameters depending on the model you use.

In [5]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-----------------------+----------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| PARAMETER             | TYPE                                   | EXAMPLE VALUE                                                                                                                                                                                                                                                                   |
+=======================+========================================+============================================================================================================================================================================================================================================================

In [6]:
params = TextChatParameters(
    temperature=1
)

### Initialize the model

Initialize the `ModelInference` class with the previously set parameters.

In [7]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=project_id,
    params=params
)

<a id="chat"></a>
## Work with chat messages 

### Work with a simple chat message using `chat_stream`

In [8]:
messages = [
    {
        "role": "user", 
        "content": "Which Formula 1 team is partnered with IBM?"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [9]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Scuderia Ferrari, an Italian Formula One racing team, collaborates with IBM for various technological advancements in the sport. This partnership, known as "The Race to Innovate," focuses on harnessing data and AI to enhance performance and strategy during races.

### Work with chat message with `thinking` control role
The `thinking` control role enables enhanced reasoning in model responses.

In [10]:
messages = [
    {
        "role": "control",
        "content": "thinking",
    },
    {
        "role": "user", 
        "content": "Tell me a joke"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [11]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

<think>Jokes are subjective, and what makes one person laugh might not amuse another. Here's a classic, clean joke that's widely appreciated:

Why don't scientists trust atoms?

Because they make up everything!

This joke plays on the double meaning of "make up": atoms physically compose all matter, and also, they are not reliable or truthful in the sense of being deceitful or untrustworthy. It's a pun that touches on scientific themes, which could be fitting given the AI context.</think><response>### Joke:

Why don't scientists trust atoms?

**Because they make up everything!**

This joke works by exploiting the dual meaning of "make up" – atoms literally form the basis of all physical matter, and figuratively, they could be seen as being unreliable narrators since they're the building blocks of potentially false stories (or at least, not inherently informative about the bigger picture). 

I hope this brings a smile to your face! </response>

### Work with chat message with `length` control role
Using the `length` control role, you can specify how long the model response should be.

#### Short length

In [12]:
messages = [
    {
        "role": "control",
        "content": "length short",
    },
    {
        "role": "user", 
        "content": "Explain torque"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [13]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Torque, also known as the moment of force, is the rotational equivalent of linear force, describing the force that causes rotation around an axis. It's calculated as the product of the force applied and the lever arm (perpendicular distance from the axis to the line of force), or torque = force × distance from pivot. torque is crucial in mechanics, engineering, and physics for analyzing rotational motion.

#### Long length

In [14]:
messages = [
    {
        "role": "control",
        "content": "length long",
    },
    {
        "role": "user", 
        "content": "Explain torque"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [15]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Torque, often symbolized by the Greek letter tau (τ), is a fundamental concept in physics, specifically in the field of rotational mechanics. It's the rotational equivalent of linear force in translational (straight-line) motion. Just as linear force causes an object to accelerate in the direction of the force, torque causes an object to rotate around an axis.

### Definition:
Torque is defined as the twisting force that causes rotation about an axis. It depends on three things:

1. **Magnitude of the force (F)**: The greater the force applied, the greater the torque, assuming all other factors remain constant.
2. **Perpendicular distance (r) from the force's line of action to the axis of rotation**: To maximize torque, a force should be applied as far as possible from the point of rotation (this is often referred to as the moment arm).
3. **Angle (θ) between the force vector and the moment arm**: The optimal torque is generated when the force is perpendicular to the radius (θ = 90 deg

### Work with chat message with `originality` control role
The `originality` control role specifies the type of summarization of the model response.

#### Extractive originality
When using `extractive` option, the model will create a summary of a given topic using the exact words used in its input data and may also cite its references.

In [16]:
messages = [
    {
        "role": "control",
        "content": "originality extractive",
    },
    {
        "role": "user", 
        "content": "Mineral composition of granites"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [17]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Granites are igneous rocks composed primarily of quartz, feldspar, and mica minerals. The mineral composition can vary, but common minerals found in granites include:

1. Quartz (SiO2): usually 20-25%
2. Alkali feldspar (NaAlSi3O8 or KAlSi3O8): typically 15-25%
3. Plagioclase feldspar (NaAlSi3O8-CaAl2Si2O8): constituting 35-55%
4. Biotite (K(Mg,Fe)3AlSi3O10(OH)2): generally 3-9%
5. Muscovite (KAl2(AlSi3O10)(OH)2): 1-6%
6. Amphibole (such as hornblende): occasionally present
7. Garnet (rarely, as an accessory mineral)
8. Zircon (ZrSiO4): often found as a trace mineral

These percentages can change depending on the specific granite type and its locale. Granites can also contain other accessory minerals, such as pyroxene, ilmenite, magnetite, and apatite, in small quantities. Given the wide variation, each granite's mineral composition is relatively unique, contributing to the diversity seen in granite outcrops worldwide.

#### Abstractive originality
When using `abstractive` originality, the model will paraphrase its data in order to create its response.

In [18]:
messages = [
    {
        "role": "control",
        "content": "originality abstractive",
    },
    {
        "role": "user", 
        "content": "Mineral composition of granites"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [19]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Granite is an igneous rock composed mainly of quartz, feldspar (both plagioclase and alkali feldspar), and mica (biotite and muscovite). It may also contain minor amounts of other minerals such as amphiboles, pyroxenes, and garnet. The predominant feldspar in granite can occur as alkali feldspar (rich in potassium and sodium) or plagioclase feldspar (rich in calcium and sodium). Granite's specific mineral composition varies based on its origin and cooling history, leading to diverse categories of granite. 

Here’s a general breakdown of its common mineral constituents (approximate amounts):

1. **Quartz**: 20-40% - Qartz is a hard, transparent to translucent mineral composed of silicon dioxide (SiO2). It often provides granite with its white, pink, or gray color.

2. **Feldspars**: 35-65% - 
   - **Plagioclase Feldspar**: 20-50% (rich in calcium and/or sodium) - Plagioclase feldspar, specifically andesine and bytownite, gives granite a darker, often grayish or blackish, color. 
   - **

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Rafał Chrzanowski**, Software Engineer Intern at watsonx.ai.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.